<a href="https://colab.research.google.com/github/adelacvg/ttts/blob/v4/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b v4 https://github.com/adelacvg/ttts.git

Cloning into 'ttts'...
remote: Enumerating objects: 1050, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1050 (delta 121), reused 130 (delta 109), pack-reused 892
Receiving objects: 100% (1050/1050), 72.07 MiB | 11.42 MiB/s, done.
Resolving deltas: 100% (618/618), done.
Updating files: 100% (209/209), done.


In [2]:
%cd ttts
!pip install -e .

/content/ttts
Obtaining file:///content/ttts
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ttts


In [3]:
!git lfs install
!git clone https://huggingface.co/adelacvg/TTTS_v4

Updated git hooks.
Git LFS initialized.
Cloning into 'TTTS_v4'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (13/13), 4.17 KiB | 853.00 KiB/s, done.
Filtering content: 100% (3/3), 4.01 GiB | 19.59 MiB/s, done.


In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pypinyin einops omegaconf==2.0.6 faiss-cpu==1.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.1 MB/s eta 0:00:00


In [7]:
from pypinyin import lazy_pinyin, Style
import torch
import numpy as np
from ttts.utils import vc_utils

MODELS = {
    'vqvae.pth':'TTTS_v4/model-283.pt',
}
from ttts.gpt.voice_tokenizer import VoiceBpeTokenizer
import torch.nn.functional as F
cond_audio = 'ttts/6.wav'

device = 'cuda:0'
text = "大家好，今天来点大家想看的东西。"
pinyin = ' '.join(lazy_pinyin(text, style=Style.TONE3, neutral_tone_with_five=True))
pinyin = ' '+pinyin+' '
tokenizer = VoiceBpeTokenizer('ttts/gpt/gpt_tts_tokenizer.json')
text_tokens = torch.IntTensor(tokenizer.encode(pinyin)).unsqueeze(0).to(device)
text_tokens = F.pad(text_tokens, (0, 1))  # This may not be necessary.
text_tokens = text_tokens.to(device)
text_length = torch.LongTensor([text_tokens.shape[1]]).to(device)

from ttts.utils.infer_utils import load_model
import torchaudio
import json
import torchaudio.functional as F
from ttts.utils.data_utils import spectrogram_torch,HParams
hps = HParams(**json.load(open('ttts/vqvae/config_v3.json')))
wav, sr = torchaudio.load(cond_audio)
if wav.shape[0] > 1:
    wav = wav[0].unsqueeze(0)
wav = wav.to(device)
wav32k = F.resample(wav, sr, 32000)
wav32k = wav32k[:,:int(hps.data.hop_length * (wav32k.shape[-1]//hps.data.hop_length))]
wav = torch.clamp(wav32k, min=-1.0, max=1.0)
wav_length = torch.LongTensor([wav.shape[1]])
spec = spectrogram_torch(wav, hps.data.filter_length,
                    hps.data.hop_length, hps.data.win_length, center=False).squeeze(0)
spec_length = torch.LongTensor([
    x//hps.data.hop_length for x in wav_length]).to(device)
vqvae = load_model('vqvae', MODELS['vqvae.pth'], 'ttts/vqvae/config_v3.json', device)
with torch.no_grad():
    wav_out = vqvae.infer(text_tokens, text_length, spec.unsqueeze(0), spec_length)
torchaudio.save('gen_.wav', wav_out.squeeze(0).cpu(), 32000)

In [8]:
from IPython.display import Audio
wav = wav_out.detach().cpu()
Audio(wav[0],rate=32000)